In [3]:
'''
Reader for EDF+ files.
TODO:
 - add support for log-transformed channels:
   http://www.edfplus.info/specs/edffloat.html and test with
   data generated with
   http://www.edfplus.info/downloads/software/NeuroLoopGain.zip.
 - check annotations with Schalk's Physiobank data.
Copyright (c) 2012 Boris Reuderink.
'''

import re, datetime, operator, logging
import numpy as np
from collections import namedtuple

EVENT_CHANNEL = 'EDF Annotations'
log = logging.getLogger(__name__)

class EDFEndOfData: pass


def tal(tal_str):
  '''Return a list with (onset, duration, annotation) tuples for an EDF+ TAL
  stream.
  '''
  exp = '(?P<onset>[+\-]\d+(?:\.\d*)?)' + \
    '(?:\x15(?P<duration>\d+(?:\.\d*)?))?' + \
    '(\x14(?P<annotation>[^\x00]*))?' + \
    '(?:\x14\x00)'

  def annotation_to_list(annotation):
    return unicode(annotation, 'utf-8').split('\x14') if annotation else []

  def parse(dic):
    return (
      float(dic['onset']),
      float(dic['duration']) if dic['duration'] else 0.,
      annotation_to_list(dic['annotation']))

  return [parse(m.groupdict()) for m in re.finditer(exp, tal_str)]


def edf_header(f):
  h = {}
  assert f.tell() == 0  # check file position
  assert f.read(8) == '0       '

  # recording info)
  h['local_subject_id'] = f.read(80).strip()
  h['local_recording_id'] = f.read(80).strip()

  # parse timestamp
  (day, month, year) = [int(x) for x in re.findall('(\d+)', f.read(8))]
  (hour, minute, sec)= [int(x) for x in re.findall('(\d+)', f.read(8))]
  h['date_time'] = str(datetime.datetime(year + 2000, month, day,
    hour, minute, sec))

  # misc
  header_nbytes = int(f.read(8))
  subtype = f.read(44)[:5]
  h['EDF+'] = subtype in ['EDF+C', 'EDF+D']
  h['contiguous'] = subtype != 'EDF+D'
  h['n_records'] = int(f.read(8))
  h['record_length'] = float(f.read(8))  # in seconds
  nchannels = h['n_channels'] = int(f.read(4))

  # read channel info
  channels = range(h['n_channels'])
  h['label'] = [f.read(16).strip() for n in channels]
  h['transducer_type'] = [f.read(80).strip() for n in channels]
  h['units'] = [f.read(8).strip() for n in channels]
  h['physical_min'] = np.asarray([float(f.read(8)) for n in channels])
  h['physical_max'] = np.asarray([float(f.read(8)) for n in channels])
  h['digital_min'] = np.asarray([float(f.read(8)) for n in channels])
  h['digital_max'] = np.asarray([float(f.read(8)) for n in channels])
  h['prefiltering'] = [f.read(80).strip() for n in channels]
  h['n_samples_per_record'] = [int(f.read(8)) for n in channels]
  f.read(32 * nchannels)  # reserved

  assert f.tell() == header_nbytes
  return h


class BaseEDFReader:
  def __init__(self, file):
    self.file = file


  def read_header(self):
    self.header = h = edf_header(self.file)

    # calculate ranges for rescaling
    self.dig_min = h['digital_min']
    self.phys_min = h['physical_min']
    phys_range = h['physical_max'] - h['physical_min']
    dig_range = h['digital_max'] - h['digital_min']
    assert np.all(phys_range > 0)
    assert np.all(dig_range > 0)
    self.gain = phys_range / dig_range


  def read_raw_record(self):
    '''Read a record with data and return a list containing arrays with raw
    bytes.
    '''
    result = []
    for nsamp in self.header['n_samples_per_record']:
      samples = self.file.read(nsamp * 2)
      if len(samples) != nsamp * 2:
        raise EDFEndOfData
      result.append(samples)
    return result


  def convert_record(self, raw_record):
    '''Convert a raw record to a (time, signals, events) tuple based on
    information in the header.
    '''
    h = self.header
    dig_min, phys_min, gain = self.dig_min, self.phys_min, self.gain
    time = float('nan')
    signals = []
    events = []
    for (i, samples) in enumerate(raw_record):
      if h['label'][i] == EVENT_CHANNEL:
        ann = tal(samples)
        time = ann[0][0]
        events.extend(ann[1:])
        # print(i, samples)
        # exit()
      else:
        # 2-byte little-endian integers
        dig = np.fromstring(samples, '<i2').astype(np.float32)
        phys = (dig - dig_min[i]) * gain[i] + phys_min[i]
        signals.append(phys)

    return time, signals, events


  def read_record(self):
    return self.convert_record(self.read_raw_record())


  def records(self):
    '''
    Record generator.
    '''
    try:
      while True:
        yield self.read_record()
    except EDFEndOfData:
      pass


def load_edf(edffile):
  '''Load an EDF+ file.
  Very basic reader for EDF and EDF+ files. While BaseEDFReader does support
  exotic features like non-homogeneous sample rates and loading only parts of
  the stream, load_edf expects a single fixed sample rate for all channels and
  tries to load the whole file.
  Parameters
  ----------
  edffile : file-like object or string
  Returns
  -------
  Named tuple with the fields:
    X : NumPy array with shape p by n.
      Raw recording of n samples in p dimensions.
    sample_rate : float
      The sample rate of the recording. Note that mixed sample-rates are not
      supported.
    sens_lab : list of length p with strings
      The labels of the sensors used to record X.
    time : NumPy array with length n
      The time offset in the recording for each sample.
    annotations : a list with tuples
      EDF+ annotations are stored in (start, duration, description) tuples.
      start : float
        Indicates the start of the event in seconds.
      duration : float
        Indicates the duration of the event in seconds.
      description : list with strings
        Contains (multiple?) descriptions of the annotation event.
  '''
  if isinstance(edffile, basestring):
    with open(edffile, 'rb') as f:
      return load_edf(f)  # convert filename to file

  reader = BaseEDFReader(edffile)
  reader.read_header()

  h = reader.header
  log.debug('EDF header: %s' % h)

  # get sample rate info
  nsamp = np.unique(
    [n for (l, n) in zip(h['label'], h['n_samples_per_record'])
    if l != EVENT_CHANNEL])
  assert nsamp.size == 1, 'Multiple sample rates not supported!'
  sample_rate = float(nsamp[0]) / h['record_length']

  rectime, X, annotations = zip(*reader.records())
  X = np.hstack(X)
  annotations = reduce(operator.add, annotations)
  chan_lab = [lab for lab in reader.header['label'] if lab != EVENT_CHANNEL]

  # create timestamps
  if reader.header['contiguous']:
    time = np.arange(X.shape[1]) / sample_rate
  else:
    reclen = reader.header['record_length']
    within_rec_time = np.linspace(0, reclen, nsamp, endpoint=False)
    time = np.hstack([t + within_rec_time for t in rectime])

  tup = namedtuple('EDF', 'X sample_rate chan_lab time annotations')
  return tup(X, sample_rate, chan_lab, time, annotations)

In [5]:
channel_available=[60,62,31,35,48,52,8,12]
win_size=10
sample_matrix_0=[] #[batch_size,channels,num_time_steps]
sample_matrix_1=[]
i_summary=0
def label(name):
    if(name[0]=='T0'):
        return 0
    if((name[0]=='T1')|(name[0]=='T2')):
        return 1
    print name
    print "problem!"
    return 0
    
for person in range(1,109): 
    print person
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)

    ### baseline
    data=load_edf("../S"+index_person+"R01.edf")
    X=data[0][channel_available]
    num_wins=int((X.shape[1]-win_size)/5)+1
    
    num_win=0
    while(num_win<num_wins):  
        sample_matrix_0.append(np.concatenate(X[:,num_win*5:num_win*5+win_size]))               
        num_win+=1
   

     ### task 1:open and close left or right fist
#     data=load_edf("../S"+index_person+"R03.edf")
#     X=data[0][channel_available]
#     time=data[3]
#     annotation=data[4]
    
#     i=0
#     i_max=X.shape[1]
#     for annot in annotation:
#         if (annot[0]==0.0):
#             while(time[i]!=0.0):
#                 i+=1
#         while(time[i]<annot[0]):
#             i+=1
#         while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
#             sample_matrix_0.append(X[:,i:i+win_size])
#             i+=5
#             if(i+win_size-1>=i_max):
#                 break
            


    #task 2 motion imagery 
    data=load_edf("../S"+index_person+"R04.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
    data=load_edf("../S"+index_person+"R08.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("../S"+index_person+"R12.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]

    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            if (label(annot[2])==0):
                sample_matrix_0.append(np.concatenate(X[:,i:i+win_size]))
            elif (label(annot[2])==1):
                sample_matrix_1.append(np.concatenate(X[:,i:i+win_size]))
                
            i+=5
            if(i+win_size-1>=i_max):
                break             

1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:130: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


In [8]:
sample_matrix=sample_matrix_0+sample_matrix_1
sample_matrix=np.array(sample_matrix)
random_index=range(sample_matrix.shape[0])
np.random.shuffle(random_index)
sample_matrix=sample_matrix[random_index]

In [9]:
#PCA 

from sklearn.decomposition import PCA
pca=PCA(n_components=15)
pca.fit(sample_matrix)
    

PCA(copy=True, iterated_power='auto', n_components=15, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [10]:
sample_pca_0=pca.transform(sample_matrix_0)
sample_pca_1=pca.transform(sample_matrix_1)

In [11]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
#convolution only within the dimension of timesteps (not with the channels)
#such method may work as a smoother...

batch_size=1000
num_time_steps=20
labels=tf.placeholder(tf.int32,shape=[1000])
inputs=tf.placeholder(tf.float32,shape=(1000,15))
W=tf.get_variable(name='W',shape=[15,2],dtype=tf.float32)
logits=tf.matmul(inputs,W)
#loss and train
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=labels, logits=logits)
#training loss with lda regularisation
train_loss = (tf.reduce_sum(crossent)/batch_size)

# Optimization
params = tf.trainable_variables()# the list of params to be trained
print params #check the trainable params, including encoder's ?

optimizer = tf.train.AdamOptimizer(learning_rate=0.001,epsilon=0.001)
gradients=optimizer.compute_gradients(
    train_loss,
    var_list=None,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)

update= optimizer.apply_gradients(gradients)


tf.summary.scalar('LLLLoss', train_loss)
for i,g in enumerate(gradients):
     tf.summary.histogram('GGGGradients'+str(i), g[0])
merged = tf.summary.merge_all()
sess=tf.Session()
train_writer = tf.summary.FileWriter('./summary',sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

[<tf.Variable 'W:0' shape=(15, 2) dtype=float32_ref>]


In [12]:
sess.run(init)

num_0=sample_pca_0.shape[0]
num_1=sample_pca_1.shape[0]
num_window=num_0+num_1
num_batch=int(num_window/batch_size)#rest data for test

for i in range(num_batch):
    
    #sampling training 
    random_index=range(num_0)
    np.random.shuffle(random_index)
    batch_0=sample_pca_0[random_index[:int(batch_size/2)]]
    
    random_index=range(num_1)
    np.random.shuffle(random_index)
    batch_1=sample_pca_1[random_index[:int(batch_size/2)]]
    
    batch=np.concatenate((batch_0,batch_1),axis=0)
    random_index=range(batch_size)
    np.random.shuffle(random_index)
    batch=batch[random_index]
    #batch=np.expand_dims(batch,axis=3)
    
    batch_labels=[]
    for i in random_index:
        if i < int(batch_size/2):
            batch_labels.append(0)
        else :
            batch_labels.append(1)

    
    #run
    _,summary=sess.run([update,merged],feed_dict={labels:batch_labels,inputs:batch})
    train_writer.add_summary(summary, i_summary)
    i_summary+=1


In [15]:
#test
result=[]
labels_test=[]
for i in range(5):
    
    #sampling training 
    random_index=range(num_0)
    np.random.shuffle(random_index)
    batch_0=sample_pca_0[random_index[:int(batch_size/2)]]
    
    random_index=range(num_1)
    np.random.shuffle(random_index)
    batch_1=sample_pca_1[random_index[:int(batch_size/2)]]
    
    batch=np.concatenate((batch_0,batch_1),axis=0)
    random_index=range(batch_size)
    np.random.shuffle(random_index)
    batch=batch[random_index]
    
    batch_labels=[]
    for i in random_index:
        if i < int(batch_size/2):
            batch_labels.append(0)
        else :
            batch_labels.append(1)
    labels_test+=batch_labels
    #run 
    r=sess.run([logits],{inputs:batch})
    result+=r
print result[0]
    
result=np.concatenate(result,axis=0)
result=np.array(result)
result=np.argmax(result,axis=1)
labels_test=np.array(labels_test)
from sklearn.metrics import classification_report
print classification_report(labels_test,result)

[[ 214.76302   214.81348 ]
 [ 152.48497   152.24101 ]
 [ 118.83481   118.6343  ]
 ...
 [  28.937862   28.740828]
 [-136.33104  -136.04376 ]
 [ 221.97063   221.6375  ]]
             precision    recall  f1-score   support

          0       0.54      0.51      0.52      2500
          1       0.53      0.55      0.54      2500

avg / total       0.53      0.53      0.53      5000

